In [36]:
# Representation Learning - November 2017

import numpy as np
import scipy as sp
import scipy.optimize as opt
import numpy.linalg as alg
from math import pi

In [127]:
# Sets parameters and constructs data set

set_size = 100
Y_dim = 10
sigma = 0.1

A = np.random.uniform(0, 1, size = (Y_dim,2))
I = np.eye(Y_dim)
O1 = np.ones((Y_dim, 1))
O2 = np.transpose(O1)

X = np.linspace(0, 4*pi, set_size)
Y = np.zeros((set_size, Y_dim))

for i in range(set_size):
    Y[i][:] = f_lin(f_non_lin(X[i]))

B = np.array([[1, 7], [3, 42]])


[1 3]


In [28]:
# Creates functions

def f_non_lin(x):
    result = np.array([x*np.sin(x), x*np.cos(x)])
    return result

def f_lin(x):
    x = x.reshape(-1,1)
    result = np.dot(np.transpose(x),np.transpose(A))
    return result

In [153]:
# Optimization part - Assuming f is a R to R^10 mapping

def fun(W, *args):
    
    # Be sure that all vectors have matrix dimensions
    Y = args[0]
    W = W.reshape(-1,1)
    Y = Y.reshape(-1,1)
    #print(Y.shape)
    #print(W.shape)
    
    matrix = np.dot(W, np.transpose(W)) + sigma**2*I
    result = 0.5*Y_dim*np.log(2*pi) + 0.5*np.log(alg.det(matrix)) + 0.5*np.dot(np.transpose(Y), np.dot(alg.inv(matrix), Y))
    result = result[0][0]
    return result
    
def gradient_fun(W, *args):
    
    # Be sure that all vectors have matrix dimensions
    Y = args[0]
    W = W.reshape(-1,1)
    Y = Y.reshape(-1,1)
    #print(Y.shape)
    #print(W.shape)
    
    matrix = np.dot(W, np.transpose(W)) + sigma**2*I
    matrix_inv = alg.inv(matrix)
    one = np.dot(O1, np.transpose(W)) + np.dot(W, O2)
    
    result = np.zeros(Y_dim)
    
    for i in range(Y_dim):
        K = np.zeros((Y_dim, Y_dim))
        W = np.reshape(W, Y_dim)
        K[i,:] += W
        K[:,i] += W
        W = W.reshape(-1,1)
        result[i] = 0.5*np.trace(np.dot(matrix_inv, K)) - 0.5*np.dot(np.transpose(Y), np.dot(matrix_inv, np.dot(K, np.dot(matrix_inv, Y))))
    
    # WARNING result should be a vector since it is the gradient of a scalar function 
    return result

In [154]:
# Perfoms optimization

x0 = np.ones(10)
args = (Y[1], Y[1])

opt.fmin_cg(fun, x0, fprime = gradient_fun, args = args)

         Current function value: -12.307323
         Iterations: 9
         Function evaluations: 38
         Gradient evaluations: 26


array([ 0.0946543 ,  0.12007603,  0.03616506,  0.07374981,  0.04893288,
        0.09724907,  0.09342524,  0.07636402,  0.04259373,  0.10741454])